In [15]:
from dask import dataframe
from dask import delayed #decorador para indicar procesamiento en paralelo
from sklearn import linear_model 
from sklearn.model_selection import train_test_split
import pandas as pd
from time import time
from scipy.stats import randint as sp_randint
from scipy import stats
from distributed import Client
import distributed.joblib
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression
import timeit
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection as ms
from sklearn.linear_model import Ridge
import numpy as np

# Con Pandas DataFrame (sklearn)

In [2]:
pd_trips_df = pd.read_csv("/data/trips.csv")
pd_trips_df.tpep_pickup_datetime = pd_trips_df.tpep_pickup_datetime.astype('M8[us]')
pd_trips_df.tpep_dropoff_datetime = pd_trips_df.tpep_dropoff_datetime.astype('M8[us]')
pd_trips_df["trip_duration"] = pd_trips_df.tpep_dropoff_datetime - pd_trips_df.tpep_pickup_datetime
pd_trips_df.trip_duration.map(lambda d: d.total_seconds()).head()

0    1170.0
1     587.0
2     627.0
3     551.0
4     568.0
Name: trip_duration, dtype: float64

In [3]:
pd_trips_df["prop"] = pd_trips_df.tip_amount / pd_trips_df.fare_amount
pd_trips_df.dtypes

car_type                          object
fare_amount                      float64
passenger_count                    int64
taxi_id                            int64
tip_amount                       float64
tpep_dropoff_datetime     datetime64[ns]
tpep_pickup_datetime      datetime64[ns]
trip_distance                    float64
trip_duration            timedelta64[ns]
prop                             float64
dtype: object

In [4]:
pd_trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,trip_duration,prop
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30,0.209091
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47,0.000000
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27,0.133333
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11,0.117647
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28,0.221333


In [5]:
pd_trips_df["car_type"] = pd_trips_df["car_type"].astype('category')
pd_trips_df.dtypes

car_type                        category
fare_amount                      float64
passenger_count                    int64
taxi_id                            int64
tip_amount                       float64
tpep_dropoff_datetime     datetime64[ns]
tpep_pickup_datetime      datetime64[ns]
trip_distance                    float64
trip_duration            timedelta64[ns]
prop                             float64
dtype: object

In [6]:
pd_trips_df["car_type_cat"] = pd_trips_df["car_type"].cat.codes
pd_trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,trip_duration,prop,car_type_cat
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30,0.209091,0
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47,0.000000,0
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27,0.133333,0
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11,0.117647,0
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28,0.221333,0


In [7]:
# Variables explicativas: duracion del viaje y tipo de carro (categorico)
df_model = pd_trips_df[['prop','trip_distance','passenger_count']]
df_model = df_model.dropna(axis=0,how='any')
y = df_model.prop
X_train, X_test, y_train, y_test = train_test_split(df_model, y, test_size=0.2)
X_train.shape, y_train.shape

((7356, 3), (7356,))

In [8]:
X_train.head()

,prop,trip_distance,passenger_count
7332,0.250909,0.96,2
3763,0.222222,0.50,1
7681,0.000000,2.08,2
4405,0.200000,1.01,5
1347,0.125000,3.14,1


In [9]:
y_train.head()

7332    0.250909
3763    0.222222
7681    0.000000
4405    0.200000
1347    0.125000
Name: prop, dtype: float64

## Tiempo de Ejecusión Pandas DataFrame

In [36]:
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False], 'n_jobs':[10,20,30]}
lm = linear_model.LinearRegression(fit_intercept=False)
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)
predictions[0:10]
grid = GridSearchCV(model,parameters, cv=3)
grid.fit(X_train, y_train)
print("Mejor Modelo:", grid.best_estimator_,"\nMejor Score:", grid.best_score_,"\n Suma de Errores Cuadraticos:", np.mean((grid.predict(X_test) - y_test) ** 2))
%time
PD_time = %time

Mejor Modelo: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=10, normalize=True) 
Mejor Score: 1.0 
 Suma de Errores Cuadraticos: 9.6006351678501e-32
CPU times: user 0 ns, sys: 7 µs, total: 7 µs
Wall time: 12.6 µs
CPU times: user 0 ns, sys: 7 µs, total: 7 µs
Wall time: 12.2 µs


## Tiempo de Ejecusión Dask

In [38]:
from dask.distributed import Client
client = Client() 
import dask_ml.joblib
from sklearn.externals.joblib import parallel_backend
with parallel_backend('dask'):
    from sklearn.grid_search import GridSearchCV
    model = linear_model.LinearRegression()
    parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False], 'n_jobs':[10,20,30]}
    grid_dask = GridSearchCV(model,parameters, cv=3)
    grid_dask.fit(X_train, y_train)
    print("Mejor Modelo:", grid_dask.best_estimator_,"\n Mejor Score:", grid_dask.best_score_,"\n Suma de Errores Cuadraticos:", np.mean((grid_dask.predict(X_test) - y_test) ** 2))
%time
Dask_time = %time

Mejor Modelo: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=10, normalize=True) 
 Mejor Score: 1.0 
 Suma de Errores Cuadraticos: 9.6006351678501e-32
CPU times: user 16 µs, sys: 7 µs, total: 23 µs
Wall time: 42.9 µs
CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 28.8 µs
